In [0]:
%sql
-- customers table backup 
CREATE TABLE customer_backup 
USING DELTA 
LOCATION 'dbfs:/FileStore/backup/customer';

In [0]:
%sql
-- orders table backup 
CREATE TABLE orders_backup 
USING DELTA 
LOCATION 'dbfs:/FileStore/backup/orders';

In [0]:
%sql
-- order_details table backup 
CREATE TABLE order_details_backup 
USING DELTA
LOCATION 'dbfs:/FileStore/backup/order_details';

In [0]:
%sql 
-- Find the customers who have placed the orders above the average order amount 
SELECT c.first_name, c.last_name, o.total_amount
FROM customer_backup c
JOIN orders_backup o 
ON c.customer_id = o.customer_id
WHERE o.total_amount > (SELECT AVG(total_amount) 
                        FROM orders_backup);

In [0]:
%sql 
-- List the products that have been sold more than 5 times in total 
SELECT od.product_name, SUM(od.quantity)
FROM order_details_backup od 
GROUP BY od.product_name
HAVING SUM(od.quantity) > (SELECT 2);

In [0]:
%sql 
-- List the customers and their total quantity of products orderd
WITH customer_orders AS (
    SELECT c.customer_id, c.first_name, sum(od.quantity) as total_quantity
    FROM customer_backup c 
    INNER JOIN orders_backup o ON c.customer_id = o.customer_id
    INNER JOIN order_details_backup od ON od.order_id = o.order_id
    GROUP BY c.customer_id, c.first_name
 )
SELECT *
FROM customer_orders
WHERE total_quantity > 3

In [0]:
%sql 
-- Find Products with their total sales revenue (price * quantity)
WITH product_sales AS (
    SELECT product_name, SUM(price_each * quantity) total_sales
    FROM order_details_backup
    GROUP BY product_name
)

SELECT * FROM product_sales 
WHERE total_sales > 10000

## # CASE STATEMENTS (SQL IF/ELSE)

In [0]:
%sql 
-- Categorize each order: 'Small', 'Meduim', 'Large'
WITH categorized_orders AS (
    SELECT order_id, total_amount,
        CASE 
            WHEN total_amount < 1000 THEN 'SMALL'
            WHEN total_amount BETWEEN 1000 and 3000 THEN 'MEDIUM'
            WHEN total_amount > 3000 THEN 'LARGE'
        END AS ORDER_SIZE
    FROM orders_backup
)
SELECT * 
FROM categorized_orders
WHERE ORDER_SIZE = 'LARGE'

In [0]:
%sql
-- Label the customers as 'New' or 'Returning' based on the order count 
SELECT c.customer_id, c.first_name, COUNT(o.order_id) order_count,
    CASE 
        WHEN order_count = 1 THEN 'NEW'
        WHEN order_count > 1 THEN 'RETURNING'
        ELSE 'No Orders'
    END AS customer_status 
FROM customer_backup c 
LEFT JOIN orders_backup o ON c.customer_id = o.customer_id
GROUP BY c.customer_id, c.first_name

## FUNCTIONS (STRING + DATE)

In [0]:
%sql
SELECT UPPER(first_name) AS FIRST_NAME_UPPERCASE 
FROM customer_backup

In [0]:
%sql
SELECT LOWER(first_name) AS FIRST_NAME_UPPERCASE 
FROM customer_backup 

In [0]:
%sql
SELECT UPPER(CONCAT(first_name, ' ' ,last_name)) AS FULL_NAME 
FROM customer_backup

In [0]:
%sql 
-- EXTRACT Month of each order & group order by month 
SELECT MONTH(order_date) AS ORDER_MONTH , COUNT(*) as TOTAL_ORDERS
FROM orders_backup
GROUP BY ORDER_MONTH 
ORDER BY ORDER_MONTH ASC 

In [0]:
%sql 
-- List each customer's name + their total spend + spending category 
WITH customer_spend AS (
    SELECT c.customer_id,
    CONCAT(c.first_name,' ',c.last_name) AS FULLNAME,
    SUM(od.quantity * od.price_each) AS TOTAL_SPEND
    FROM customer_backup c 
    INNER JOIN orders_backup o ON c.customer_id = o.customer_id
    INNER JOIN order_details_backup od ON o.order_id = od.order_id
    GROUP BY c.customer_id, FULLNAME
)
SELECT FULLNAME, TOTAL_SPEND,
      CASE 
          WHEN TOTAL_SPEND < 10000 THEN 'BRONZE'
          WHEN TOTAL_SPEND BETWEEN 1000 AND 30000 THEN 'SILVER'
          WHEN TOTAL_SPEND > 30000 THEN 'GOLD'
      END AS CUSTOMER_TIER
FROM customer_spend;